In [ ]:
import os
import pyarrow.parquet as pq
import pandas as pd

In [5]:
#sample raw data
raw_data = pq.read_pandas("Documents/Transportation Data/fhvhv_tripdata_2024-07.parquet") #reading parquet file
raw_data = raw_data.to_pandas()
raw_data.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2024-07-01 00:13:16,2024-07-01 00:18:28,2024-07-01 00:19:43,2024-07-01 00:40:35,138,141,8.84,...,4.70,2.75,2.5,9.28,24.19,N,N,N,N,N
1,HV0003,B03404,B03404,2024-07-01 00:16:44,2024-07-01 00:18:58,2024-07-01 00:21:00,2024-07-01 00:41:24,61,80,3.96,...,2.22,0.00,0.0,0.00,19.37,N,N,N,N,N
2,HV0003,B03404,B03404,2024-07-01 00:38:38,2024-07-01 00:45:10,2024-07-01 00:45:10,2024-07-01 01:01:05,36,260,5.61,...,1.84,0.00,0.0,0.00,12.08,Y,Y,N,N,N
3,HV0003,B03404,B03404,2024-07-01 00:38:26,2024-07-01 00:48:38,2024-07-01 00:49:01,2024-07-01 01:15:02,80,42,11.24,...,4.05,0.00,0.0,0.00,25.64,Y,Y,N,N,N
4,HV0003,B03404,B03404,2024-07-01 00:34:57,2024-07-01 00:40:58,2024-07-01 00:41:36,2024-07-01 00:49:48,152,41,1.60,...,1.03,0.00,0.0,5.00,7.11,N,N,N,N,N


In [1]:
#this method creates a new data frame containing the average base fare, average total fare (sum of base fare, surcharge
# tips, tolls, salestax, and airport fees) for uber and lyft rides in a given month and year
def creating_clean_data(file_path):
  trips = pq.read_pandas(file_path) #reading parquet file
  trips = trips.to_pandas()  
  trips = trips[trips['hvfhs_license_num'].isin(['HV0003', 'HV0005'])] #only keeping uber (HV003) and lyft (HV005) data
  trips['hvfhs_license_num'] = trips['hvfhs_license_num'].replace({'HV0003': 'Uber', 'HV0005': 'Lyft'}) #renaming data

# adding total fare column (sum of base fare and additional charges)
  trips['total_fare'] = trips['base_passenger_fare'] + trips['tolls'] + trips['sales_tax'] + trips['congestion_surcharge'] + trips['airport_fee'] + trips['tips']

#finding the average total fare for uber and lyft
  uber_avg_tfare = trips[trips['hvfhs_license_num'] == 'Uber']['total_fare'].mean()
  lyft_avg_tfare = trips[trips['hvfhs_license_num'] == 'Lyft']['total_fare'].mean()

#finding the average base fare for uber and lyft
  uber_avg_bfare = trips[trips['hvfhs_license_num'] == 'Uber']['base_passenger_fare'].mean()
  lyft_avg_bfare = trips[trips['hvfhs_license_num'] == 'Lyft']['base_passenger_fare'].mean()

#storing the month and year of the file
  month = trips['request_datetime'].iloc[0].month
  year= trips['request_datetime'].iloc[0].year

#creating new data frame containing the month, year, and average uber and lyft total fare and base fare for that month
  avg_fares_df = pd.DataFrame({
      'Month': [month],
      'Year': [year],
      'Average Uber Total Fare': [uber_avg_tfare],
      'Average Uber Base Fare': [uber_avg_bfare],
      'Average Lyft Total Fare': [lyft_avg_tfare],
      'Average Lyft Base Fare': [uber_avg_bfare]
  })

  return avg_fares_df

#this method process all .parquet files in a directory and creates a new data frame 
# with the creating_clean_data information
def process_directory(directory_path):
  all_avg_fares = pd.DataFrame()  # Initialize an empty DataFrame to store results

  for filename in os.listdir(directory_path):
    if filename.endswith(".parquet"):
      file_path = os.path.join(directory_path, filename)
      try:
        avg_fares_df = creating_clean_data(file_path)
        all_avg_fares = pd.concat([all_avg_fares, avg_fares_df], ignore_index=True)
      except Exception as e:
        print(f"Error processing {filename}: {e}")

  return all_avg_fares

In [3]:
directory= "Documents/Transportation Data"
all_fares_df= process_directory(directory)
all_fares_df.to_csv('all_fares.csv', index=False)

In [7]:
#removed null values
all_fares_clean= all_fares_df.dropna()
all_fares_clean.head()

,Month,Year,Average Uber Total Fare,Average Uber Base Fare,Average Lyft Total Fare,Average Lyft Base Fare
0,2,2021,23.891282,19.209456,24.916026,19.209456
1,2,2024,30.003911,24.537868,28.991807,24.537868
2,9,2023,32.927748,26.916275,29.176778,26.916275
4,10,2019,21.653333,18.084945,74.210000,18.084945
5,6,2022,31.411499,25.491931,29.613316,25.491931


In [8]:
all_fares_clean.to_csv('all_fares_cleaned.csv', index=False)